In [2]:
import torch
from transformers import pipeline
import nltk

from transformers import AutoModelForSequenceClassification
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer, AutoConfig
import numpy as np

import csv

from datasets import load_dataset, DatasetDict, Dataset, concatenate_datasets
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig, DataCollatorWithPadding, AdamW, get_scheduler
from torch.utils.data import DataLoader

from torch import nn

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/khanh/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [28]:
tokenizer_0 = AutoTokenizer.from_pretrained("khanhtq2802/thesis-model")
config_0 = AutoConfig.from_pretrained("khanhtq2802/thesis-model")
model_0 = AutoModelForSequenceClassification.from_pretrained("khanhtq2802/thesis-model", force_download=False)

tokenizer_1 = AutoTokenizer.from_pretrained("VietAI/envit5-translation")
config_1 = AutoConfig.from_pretrained("VietAI/envit5-translation")
model_1 = AutoModelForSeq2SeqLM.from_pretrained("VietAI/envit5-translation")

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model_0.to(device)
model_1.to(device)
device

device(type='cuda')

seoul

In [92]:
m = nn.Softmax(dim=1)

with open("../data/seoul.csv", 'r') as csvfile:
    # skip the head
    csv_reader = csv.reader(csvfile)
    next(csv_reader, None)
    # write file
    with open("../data/seoul-after-khanhtq2802-thesis-model.csv", 'w') as csvfile1:
        fieldnames = ['en_text', 'vn_text', 'confidence', 'bot_labels']
        writer = csv.DictWriter(csvfile1, fieldnames=fieldnames)
        writer.writeheader()
        for row in csv_reader:
            for sentence in nltk.sent_tokenize(row[0]):
                try:
                    # get en_text
                    en_text = 'en: ' + sentence
                    # get vn_text
                    inputs = [en_text]
                    outputs = model_1.generate(tokenizer_1(inputs, return_tensors="pt", padding=True).input_ids.to('cuda'), max_length=150)
                    vn_text = tokenizer_1.batch_decode(outputs, skip_special_tokens=True)[0]
                    # get bot_labels and confidence
                    aspects = ['', 'amn-', 'ch-', 'ppl-', 'mgt-', 'nat-', 'prc-']
                    aspects_full = ['generality: ', 'amenities: ', 'cultural heritage: ', 'people: ', 'management: ', 'nature: ', 'price: ']
                    inputs = []
                    for i in aspects_full:
                        inputs.append(i + sentence)

                    encoded_inputs = tokenizer_0(inputs, return_tensors='pt', padding=True)
                    encoded_inputs.to(device)
                    outputs = model_0(**encoded_inputs)
                    outputs = outputs.logits

                    softmax = nn.Softmax(dim=1)(outputs)
                    pred_labels = torch.argmax(outputs, dim=1)

                    bot_labels = str(pred_labels[0].item())
                    confidence = softmax[0][pred_labels[0].item()]

                    for i in range(1, len(pred_labels)):
                        confidence += softmax[i][pred_labels[i].item()]
                        if pred_labels[i] != 0:
                            bot_labels += ' ' + aspects[i] + str(pred_labels[i].item())
                    confidence = str(confidence.item() / len(pred_labels))

                    writer.writerow({
                    'en_text': en_text,
                    'vn_text': vn_text,
                    'confidence': confidence,
                    'bot_labels': bot_labels,
                    })
                except Exception:
                    continue

beijing

In [93]:
m = nn.Softmax(dim=1)

with open("../data/beijing.csv", 'r') as csvfile:
    # skip the head
    csv_reader = csv.reader(csvfile)
    next(csv_reader, None)
    # write file
    with open("../data/beijing-after-khanhtq2802-thesis-model.csv", 'w') as csvfile1:
        fieldnames = ['en_text', 'vn_text', 'confidence', 'bot_labels']
        writer = csv.DictWriter(csvfile1, fieldnames=fieldnames)
        writer.writeheader()
        for row in csv_reader:
            for sentence in nltk.sent_tokenize(row[0]):
                try:
                    # get en_text
                    en_text = 'en: ' + sentence
                    # get vn_text
                    inputs = [en_text]
                    outputs = model_1.generate(tokenizer_1(inputs, return_tensors="pt", padding=True).input_ids.to('cuda'), max_length=150)
                    vn_text = tokenizer_1.batch_decode(outputs, skip_special_tokens=True)[0]
                    # get bot_labels and confidence
                    aspects = ['', 'amn-', 'ch-', 'ppl-', 'mgt-', 'nat-', 'prc-']
                    aspects_full = ['generality: ', 'amenities: ', 'cultural heritage: ', 'people: ', 'management: ', 'nature: ', 'price: ']
                    inputs = []
                    for i in aspects_full:
                        inputs.append(i + sentence)

                    encoded_inputs = tokenizer_0(inputs, return_tensors='pt', padding=True)
                    encoded_inputs.to(device)
                    outputs = model_0(**encoded_inputs)
                    outputs = outputs.logits

                    softmax = nn.Softmax(dim=1)(outputs)
                    pred_labels = torch.argmax(outputs, dim=1)

                    bot_labels = str(pred_labels[0].item())
                    confidence = softmax[0][pred_labels[0].item()]

                    for i in range(1, len(pred_labels)):
                        confidence += softmax[i][pred_labels[i].item()]
                        if pred_labels[i] != 0:
                            bot_labels += ' ' + aspects[i] + str(pred_labels[i].item())
                    confidence = str(confidence.item() / len(pred_labels))

                    writer.writerow({
                    'en_text': en_text,
                    'vn_text': vn_text,
                    'confidence': confidence,
                    'bot_labels': bot_labels,
                    })
                except Exception:
                    continue